In [ ]:
import numpy as np
import math
from collections import Counter
import sys
#import matplotlib.pyplot as plt
import networkx as nx
import time
import csv
from Pathlib import Path

import trimesh

class SpineClassifier(object):
    
    def __init__(self, name):
        self.mesh = None

    #used for when not pulling from datajoint
    def get_cgal_data_and_label_local(ob_name,labels_file,sdf_file):


        triangles_labels = []
        with open(labels_file) as csvfile:

            for row in csv.reader(csvfile):
                triangles_labels.append(int(row[0])) 


        ob = bpy.context.object


        me = ob.data

        #print("starting to hide everything")
        #iterate through all of the vertices
        verts_raw = ob.data.vertices
        #print(len(active_verts_raw))

        edges_raw = ob.data.edges

        #print(len(active_edges_raw))

        faces_raw = ob.data.polygons

        #gets a list of the unique labels
        unique_segments = list(Counter(triangles_labels).keys())


        segmentation_length = len(unique_segments) # equals to list(set(words))
        #print(segmentation_length)

        #makes a dictionary that maps the unique segments to a number from range(0,len(unique_seg))
        unique_index_dict = {unique_segments[x]:x for x in range(0,segmentation_length)}


        #print("unique_index_dict = " + str(len(unique_index_dict)))
        #print("triangle_labels = " + str(len(triangles_labels)))
        #adds all of the labels to the faces
        max_length = len(triangles_labels)

        #just iterate and add them to the faces
        #here is where need to get stats for sdf numbers


        labels_list = []
        for tri in triangles_labels:

            #assembles the label list that represents all of the faces
            labels_list.append(str(unique_index_dict[tri])) 



        #make sure in solid mode
        for area in bpy.context.screen.areas: # iterate through areas in current screen
            if area.type == 'VIEW_3D':
                for space in area.spaces: # iterate through spaces in current VIEW_3D area
                    if space.type == 'VIEW_3D': # check if space is a 3D view
                        space.viewport_shade = 'SOLID' # set the viewport shading to rendered

        bpy.ops.object.mode_set(mode='OBJECT')

        #these variables are set in order to keep the functions the same as FINAL_importing_auto_seg.py
        newname = ob.name
        print("done with cgal_segmentation")

        #----------------------now return a dictionary of the sdf values like in the older function get_sdf_dictionary
        #get the sdf values and store in sdf_labels
        sdf_labels = []
        with open(sdf_file) as csvfile:

            for row in csv.reader(csvfile):
                sdf_labels.append(float(row[0])) 


        sdf_temp_dict = {}
        labels_seen = []
        #iterate through the labels_list
        for i,label in enumerate(labels_list):
            if label not in labels_seen:
                labels_seen.append(label)
                sdf_temp_dict[label] = []

            sdf_temp_dict[label].append(sdf_labels[i])
        #print(sdf_temp_dict)

        #now calculate the stats on the sdf values for each label
        sdf_final_dict = {}
        sdf_final_dict_list = {}
        for dict_key,value in sdf_temp_dict.items():

            #just want to store the median

            sdf_final_dict_list[dict_key] = [np.min(value),np.median(value),np.max(value),np.percentile(value,10),np.percentile(value,90),np.std(value),
                                        np.percentile(value,80),np.percentile(value,70),len(value)]
            sdf_final_dict[dict_key] = np.median(value)

        #print these values to files


        return sdf_final_dict, labels_list, sdf_final_dict_list

    def find_neighbors(labels_list,current_label,verts_to_Face,faces_raw,verts_raw):
        """will return the number of neighbors that border the segment"""

        #iterate over each face with that label
        #   get the vertices of that face
        #   get all the faces that have that vertice associated with that
        #   get the labels of all of the neighbor faces, for each of these labels, add it to the neighbors 
        #list if it is not already there and doesn't match the label you are currently checking
        #   return the list 

        #get the indexes of all of the faces with that label that you want to find the neighbors for

        #OPTOMIZE
        index_list = []
        for i,x in enumerate(labels_list):
            if x == current_label:
                index_list.append(i)

        verts_checked = [] #vertices checked
        faces_checked = [] #keeps track of the faces checked
        neighbors_list = [] #list of all neighboring labels
        neighbors_shared_vert = {} #keeps track of total number of faces that are bordering label we care about for a particular neighbor label
        for index in index_list:
            #gets face
            current_face = faces_raw[index]

            #get the vertices associates with face
            vertices = current_face.vertices

            #get the faces associated with the vertices of that specific face

            for vert in vertices:
                added_this_round = []
                #will only check each vertex once
                if vert not in verts_checked:
                    verts_checked.append(vert)
                    faces_associated_vert = verts_to_Face[vert]
                    for fac in faces_associated_vert:
                        #make sure it is not a fellow face with the label who we are looking for the neighbors of
                        if (fac not in index_list):
                            #check to see if checked the the face already
                            if (fac not in faces_checked):
                                if(labels_list[fac] not in neighbors_list):
                                    #add the vertex to the count of shared vertices
                                    neighbors_shared_vert[labels_list[fac]] = 0 
                                    #only store the faces that are different
                                    neighbors_list.append(labels_list[fac])
                                    #faces_to_check.append(fac)
                                    #faces_to_check.insert(0, fac)
                                #increment the number of times we have seen that label face
                                if labels_list[fac] not in added_this_round:
                                    neighbors_shared_vert[labels_list[fac]] = neighbors_shared_vert[labels_list[fac]] + 1 #OPTOMIZE
                                    added_this_round.append(labels_list[fac])
                                #now add the face to the checked list
                                faces_checked.append(fac)


        #number of faces of the label we care about
        number_of_faces = len(index_list)


        #Description of Return List:
        #1) neighbors_list = labels of all bordering neighbors
        #2) neighbors_shared_vert = number of faces for each bordering neighbor
        #3) number_of_faces = total number of faces for current label
        return neighbors_list,neighbors_shared_vert,number_of_faces


    
    #generates the stats: connections on who it is connected to), shared_verts (how many vertices it shares between it's neighbor), mesh_number (number of face for that label)
    def export_connection(labels_list,label_name, verts_to_Face,outputFlag="False",file_name="None"):

        #print("hello from export_connection with label_name = " + str(label_name) )
        #find all the neighbors of the label

        currentMode = bpy.context.object.mode

        bpy.ops.object.mode_set(mode='OBJECT')
        ob = bpy.context.object
        ob.update_from_editmode()

        #print("object_name = " + bpy.context.object.name)
        me = ob.data

        faces_raw = me.polygons
        verts_raw = me.vertices

        #print("generating list in export connections")
        #labels_list = generate_labels_list(faces_raw)
        #print("done generating list in export connections")


        #need to assemble a dictionary that relates vertices to faces
        #*****making into a list if the speed is too slow*******#
        #print("about to making verts_to_Face")
        #verts_to_Face = generate_verts_to_face_dictionary(faces_raw,verts_raw)
        #print("DONE about to making verts_to_Face")

        total_labels_list = []
        faces_checked = []
        faces_to_check = [label_name]

        still_checking_faces = True

        connections = {}
        shared_vertices = {}
        mesh_number = {}

        #print("about to start checking faces")

        #will iterate through all of the labels with the label name until find all of the neighbors (until hitting the backbone) of the label
        while still_checking_faces:
            #will exit if no more faces to check
            if not faces_to_check:
                still_checking_faces = False
                break

            for facey in faces_to_check:
                if facey != "backbone":
                    neighbors_list,neighbors_shared_vert,number_of_faces = find_neighbors(labels_list,facey,verts_to_Face,faces_raw,verts_raw)



                    #reduce the shared vertices with a face and the backbone to 0 so doesn't mess up the shared vertices percentage
                    pairs = list(neighbors_shared_vert.items())
                    pre_connections = [k for k,i in pairs]
                    pre_shared_vertices = [i for k,i in pairs]




                    if ("backbone" in pre_connections):
                        back_index = pre_connections.index("backbone")
                        pre_shared_vertices[back_index] = 0


                    connections[facey] = pre_connections
                    shared_vertices[facey] = pre_shared_vertices
                    mesh_number[facey] = number_of_faces


                    for neighbors in neighbors_list:
                        if (neighbors != "backbone") and (neighbors not in faces_to_check) and (neighbors not in faces_checked):
                            faces_to_check.append(neighbors)

                    faces_to_check.remove(facey)
                    faces_checked.append(facey)

            #append the backbone to the graph structure
            mesh_number["backbone"] = 0

        #print("faces_checked = " + str(faces_checked))
        #print("DONE about to start checking faces")

        #save off the file to an npz file


        if(outputFlag == True):
            complete_path = str("/Users/brendancelii/Google Drive/Xaq Lab/Datajoint Project/Automatic_Labelers/spine_graphs/"+file_name)



            #package up the data that would go to the database and save it locally name of the file will look something like this "4_bcelii_2018-10-01_12-12-34"
        #    np.savez("/Users/brendancelii/Google Drive/Xaq Lab/Datajoint Project/local_neurons_saved/"+segment_ID+"_"+author+"_"+
        #        date_time[0:9]+"_"+date_time[11:].replace(":","-")+".npz",segment_ID=segment_ID,author=author,
        #					date_time=date_time,vertices=vertices,triangles=triangles,edges=edges,status=status)
            np.savez(complete_path,connections=connections,shared_vertices=shared_vertices,mesh_number=mesh_number ) 

        return connections,shared_vertices,mesh_number


    def relabel_segments(labels_list,current_label,new_label):
        for i,x in enumerate(labels_list):
            if x == current_label:
                labels_list[i] = new_label

        return labels_list

    def generate_verts_to_face_dictionary(faces_raw,verts_raw):
        verts_to_Face = {}

        #initialize the lookup dictionary as empty lists
        for pre_vertex in verts_raw:
            verts_to_Face[pre_vertex.index] = []

        #print(len(verts_raw))
        #print(len(verts_to_Face))
        #print(verts_to_Face[1])

        for face in faces_raw:
            #get the vertices
            verts = face.vertices
            #add the index to the list for each of the vertices
            for vertex in verts:
                verts_to_Face[vertex].append(face.index)

        return verts_to_Face


    ##Functins from the auto_spine_labeler
    def smooth_backbone_vp4(labels_list,sdf_final_dict,backbone_width_threshold = 0.35,max_backbone_threshold = 400,backbone_threshold=300,shared_vert_threshold=25,shared_vert_threshold_new = 5,backbone_neighbor_min=10,number_Flag = False, seg_numbers=1,smooth_Flag=True):
        print("at beginning of smooth backbone vp4")
        #things that could hint to backbone
        #1) larger size
        #2) touching 2 or more larger size
        #have to go into object mode to do some editing
        currentMode = bpy.context.object.mode

        bpy.ops.object.mode_set(mode='OBJECT')
        ob = bpy.context.object
        ob.update_from_editmode()

        #print("object_name = " + bpy.context.object.name)
        me = ob.data

        #print("about to get faces_verts raw")
        faces_raw = me.polygons
        verts_raw = me.vertices
        #print("DONE about to get faces_verts raw")

        #print("don't need to generate labels_list anymore")
        #print("about to generate labels_list")   ####!!!! This takes a good bit of time#####
        #labels_list = generate_labels_list(faces_raw)
        #print("DONE about to generate labels_list")

        #need to assemble a dictionary that relates vertices to faces
        #*****making into a list if the speed is too slow*******#



        #print("about to generate verts_to_Face")
        verts_to_Face = generate_verts_to_face_dictionary(faces_raw,verts_raw)
        #print("DONE about to generate verts_to_Face")
        #add new color and reassign all of the labels with those colors as the backbone label

        #create a list of all the labels and which ones are the biggest ones
        from collections import Counter


        myCounter = Counter(labels_list)

        spine_labels = []
        backbone_labels = []

        #put groups that have number of faces > max_backbone_threshold into backbone list
        for label,times in myCounter.items():
            if(times >= max_backbone_threshold):
                #print(str(label) + ":" + str(times))
                backbone_labels.append(label)   

        #put groups that have median sdf value > backbone_width_threshold AND greater than backbone_threshold into backbone list
        for label in myCounter.keys():
            if( sdf_final_dict[label] >= backbone_width_threshold):
                #print(str(label) + ":" + str(times))
                if(myCounter[label] > backbone_threshold) and (label not in backbone_labels):
                    backbone_labels.append(label)   
        #print(" DONE about to get counter list")


        to_remove = []

        backbone_neighbors_dict = {}

        #beginning smoothing round that removes ones from backbone list
        for i in range(0,5):
            print("smoothing round " + str(i+1))
            printout_counter = 0
            counter = 0
            #iterates through all the groups that were designated as backbones
            for bkbone in backbone_labels:
                if bkbone not in to_remove: #if not already designated to be removed

                    if bkbone not in backbone_neighbors_dict.keys(): #if haven't already found the neighbors for that label
                        #find_neighbors Description of Return List:
                        #1) neighbors_list = labels of all bordering neighbors
                        #2) neighbors_shared_vert = number of faces for each bordering neighbor
                        #3) number_of_faces = total number of faces for current label
                        neighbors_list,neighbors_shared_vert,number_of_faces = find_neighbors(labels_list,bkbone,verts_to_Face,faces_raw,verts_raw)
                        #add the neighbor stats and count to the dictionary corresponding to that label
                        backbone_neighbors_dict[bkbone] = dict(neighbors_list=neighbors_list,neighbors_shared_vert=neighbors_shared_vert,
                            number_of_faces=number_of_faces)
                    else:
                        #just retrieve the neighbor stats and count of faces that are already stored in dict
                        neighbors_list = backbone_neighbors_dict[bkbone]["neighbors_list"]
                        neighbors_shared_vert = backbone_neighbors_dict[bkbone]["neighbors_shared_vert"]
                        number_of_faces = backbone_neighbors_dict[bkbone]["number_of_faces"]

                    #counts up the number of shared vertices with backbone neighbors

                    #OPTOMIZE
                    backbone_count_flag = False
                    neighbor_counter = 0 #TOTAL NUMBER OF BACKBONE NEIGHBORS
                    #spine_neighbor_counter = 0
                    total_backbone_shared_verts = 0 #TOTAL NUMBER OF FACES SHARED WITH BACKBONE
                    for n in neighbors_list:         
                        if (n in backbone_labels) and (n not in to_remove):
                            neighbor_counter += 1
                            total_backbone_shared_verts = total_backbone_shared_verts + neighbors_shared_vert[n] 

                    interested_id = str(997)
                    if str(bkbone) == str(interested_id):
                        print(interested_id + " total_backbone_shared_verts = " + str(total_backbone_shared_verts))
                        print(interested_id + " neighbor_counter = " + str(neighbor_counter))



                    #OPTOMIZE
                    #if meets requirement of shared verts then activates flag     
                    if (total_backbone_shared_verts > shared_vert_threshold):
                        backbone_count_flag = True

                    #if there are no neighbor's that are backbones or does not share enough backbone vertices --> remove from backbone list
                    if neighbor_counter <= 0 or backbone_count_flag == False:
                            to_remove.append(bkbone)
                            counter += 1


            #if 1 or less non-backbones were converted to remove list then go ahead to the next step
            print("counter = " + str(counter))
            if counter <= 1:
                print("counter caused the break")
                break

        print("just broke out of the loop")
        """
        Status: 
        1) Started with a tentative list of backbones
        2) Removed some potential backbone lists
        """




        #now go through and make sure no unconnected backbone segments

        """Pseudo-code for filtering algorithm
        1) iterate through all of the backbone labels
        2) Go get the neighbors of the backbone
        3) Add all of the neighbors who are too part of the backbone to the backbones to check list
        4) While backbone neighbor counter is less than the threshold or until list to check is empty
        5) Pop the next neighbor off the list and add it to the neighbors check list
        6) Get the neighbors of this guy
        7) for each of neighbors that is also on the backbone BUT HASN'T BEEN CHECKED YET append them to the list to be check and update counter
        8) continue at beginning of loop
        -- once loop breaks
        9) if the counter is below the threshold:
            Add all of values in the neighbros already checked list to the new_to_remove
        10) Use the new_backbone_labels and new_to_remove to rewrite the labels_list

        """

        #gets the new backbones list without the ones removed
        new_backbone_labels = [bkbone for bkbone in backbone_labels if bkbone not in to_remove] #OPTOMIZE
        new_to_remove = []
        skip_labels = []

        print("new_backbone_labels lenght = " + str(len(new_backbone_labels)))

        for bkbonz in new_backbone_labels:
            if bkbonz not in skip_labels:
                #print("working on backbone = " + str(bkbonz))
                checked_backbone_neighbors = []
                backbone_neighbors_to_check = []
                new_backbone_neighbor_counter = 0


                if bkbonz not in backbone_neighbors_dict.keys(): #should never enter this loop..... #OPTOMIZE
                    neighbors_list,neighbors_shared_vert,number_of_faces = find_neighbors(labels_list,bkbonz,verts_to_Face,faces_raw,verts_raw)
                    backbone_neighbors_dict[bkbonz] = dict(neighbors_list=neighbors_list,neighbors_shared_vert=neighbors_shared_vert,
                        number_of_faces=number_of_faces)
                else: #gets the stats of the neighbors and count of current label
                    neighbors_list = backbone_neighbors_dict[bkbonz]["neighbors_list"]
                    neighbors_shared_vert = backbone_neighbors_dict[bkbonz]["neighbors_shared_vert"]
                    number_of_faces = backbone_neighbors_dict[bkbonz]["number_of_faces"]

                for bb in neighbors_list:
                    #counts as viable backbone neighbor if meets following conditions:
                    #1) In the new backbone list
                    #2) hasn't been checked yet
                    #3) not in the new ones to remove
                    #4) The number of neighbors shared by that label is greater than raw threshold shared_vert_threshold_new

                    #OPTOMIZE: don't need checked_backbone_neighbors
                    if (bb in new_backbone_labels) and (bb not in checked_backbone_neighbors) and (bb not in new_to_remove) and neighbors_shared_vert[bb] > shared_vert_threshold_new:
                        backbone_neighbors_to_check.append(bb)
                        new_backbone_neighbor_counter += 1

                #at this point have :
                #1) total number of backbone neighbors: new_backbone_neighbor_counter
                #2) backbone neighbors in list: backbone_neighbors_to_check

                checked_backbone_neighbors = [nb for nb in backbone_neighbors_to_check]


                #4) While backbone neighbor counter is less than the threshold or until list to check is empty

                #Iterates through all possible backbone neighbors unitl:
                # A) new_backbone_neighbor_counter is greater than set threshold of backbone_neighbor_min OR
                # B) no more backbone neighbors to check

                #Goal: counts the backbone chain with that label, so in hopes if not high enough then not backbone piece
                while new_backbone_neighbor_counter < backbone_neighbor_min and backbone_neighbors_to_check != []:
                    #5) Pop the next neighbor off the list and add it to the neighbors check list
                    current_backbone = backbone_neighbors_to_check.pop(0)
                    if current_backbone not in checked_backbone_neighbors:
                        checked_backbone_neighbors.append(current_backbone) #mark it as checked
                    #6) Get the neighbors of this guy
                    if current_backbone not in backbone_neighbors_dict.keys(): #should already be in there
                        neighbors_list,neighbors_shared_vert,number_of_faces = find_neighbors(labels_list,current_backbone,verts_to_Face,faces_raw,verts_raw)
                        backbone_neighbors_dict[current_backbone] = dict(neighbors_list=neighbors_list,neighbors_shared_vert=neighbors_shared_vert,
                            number_of_faces=number_of_faces)
                    else: #gets the current neighbors and counts of one of the possible neighbor backbones
                        neighbors_list = backbone_neighbors_dict[current_backbone]["neighbors_list"]
                        neighbors_shared_vert = backbone_neighbors_dict[current_backbone]["neighbors_shared_vert"]
                        number_of_faces = backbone_neighbors_dict[current_backbone]["number_of_faces"]

                    #7) for each of neighbors that is also on the backbone BUT HASN'T BEEN CHECKED YET append them to the list to be check and update counter
                    for bb in neighbors_list:
                        if (bb in new_backbone_labels) and (bb not in checked_backbone_neighbors) and (bb not in new_to_remove) and neighbors_shared_vert[bb] > shared_vert_threshold_new:
                            backbone_neighbors_to_check.append(bb)
                            new_backbone_neighbor_counter += 1

                #9) if the counter is below the threshold --> Add all of values in the neighbros already checked list to the new_to_remove
                if new_backbone_neighbor_counter < backbone_neighbor_min:
                    for bz in checked_backbone_neighbors:
                        if bz not in new_to_remove:
                            new_to_remove.append(bz)
                            print("removed " + str(checked_backbone_neighbors))
                else:
                    skip_labels = skip_labels + checked_backbone_neighbors



        print("done Analyzing big and small segments")        
        #go through and switch the label of hte 
        #may not want to relabel until the end in order to preserve the labels in case label a big one wrong
        print("about to rewrite the labels")
        for i in range(0,len(labels_list)):
            if labels_list[i] in new_backbone_labels and labels_list[i] not in new_to_remove:
                labels_list[i] = "backbone"
                faces_raw[i].material_index = 0
            else:
                faces_raw[i].material_index = 2




        print("DONE about to rewrite the labels")
        return labels_list, verts_to_Face

    def filter_Stubs(labels_list,verts_to_Face,sdf_final_dict):

        myCounter = Counter(labels_list)
        complete_labels =  [label for label,times in myCounter.items()]

        stub_threshold = 50

        processed_labels = []
        for i in range(0,len(complete_labels)):
            if complete_labels[i] != "backbone" and complete_labels[i] not in processed_labels:
                #print("at beginning of spine labeling loop: about to enter export connection")
                #get the conenections, shared vertices and mesh sizes for the whole spine segment in which label is connected to
                connections,shared_vertices,mesh_number = export_connection(labels_list,complete_labels[i], verts_to_Face,outputFlag="False",file_name="None")

                total_mesh_faces_outer = sum([k for i,k in mesh_number.items()])

                #once done all of the paths go through and label things as stubs
                if total_mesh_faces_outer < stub_threshold:
                    #print("stub threshold triggered")
                    for label_name in mesh_number.keys():
                        relabel_segments(labels_list,label_name,"backbone")
                for key in mesh_number.keys():
                    processed_labels.append(key)

        return labels_list


    #from w1_Visualize_whole_pre_classify import create_local_colors

    def get_spine_classification(file_location,file_name,clusters,smoothness):        
        original_start_time = time.time()    
        start_time = time.time()

        #initialize the object
        file_location = "/notebooks/18_Spine_Allen"
        file_name = "neuron-775959265587_part_2.off"
        full_path = str(Path(file_location) / Path(file_name))
        self.mesh = trimesh.load_mesh(full_path)
        

        faces_raw = obj.data.polygons
        #faces_raw = self.mesh.faces

        #create_local_colors(obj)

        print("loading object and box--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()


        labels_file = file_location + "/cgal/" + file_name + "-cgal_" + str(clusters) + "_" + str(smoothness) + ".csv" 
        sdf_file = file_location + "/cgal/" + file_name + "-cgal_" + str(clusters) + "_" + str(smoothness) + "_sdf.csv"

        sdf_final_dict, labels_list, sdf_final_dict_list = get_cgal_data_and_label_local(ob_name,labels_file,sdf_file)

        #print out the total statistics: 
        sdf_analysis_file = file_location + "/cgal_analysis/" + file_name + "-cgal_" + str(clusters) + "_" + str(smoothness) + ".csv" 
        with open(sdf_analysis_file[:-4] + "_STATS_TOTAL.csv",mode="w") as sdf_stat_file:
            sdf_writer = csv.writer(sdf_stat_file,delimiter=",")



            #[np.min(value),np.median(value),np.max(value),np.percentile(value,10),np.percentile(value,90),np.std(value)]
            sdf_writer.writerow(["label","min","median","max","10th percentile","90th percentile","std dev","80th percentile",
                                    "70th percentile","NUM OF FACES"])
            for key,value in sdf_final_dict_list.items():
                sdf_writer.writerow([str(key),*value])


        #write the special files for missed
        missed_labels = [385,761,90,593,384,537,811,287,1615,997,693]
        with open(sdf_analysis_file[:-4] + "_STATS_TOTAL_incorrect.csv",mode="w") as sdf_stat_file:
            sdf_writer = csv.writer(sdf_stat_file,delimiter=",")

            #[np.min(value),np.median(value),np.max(value),np.percentile(value,10),np.percentile(value,90),np.std(value)]
            sdf_writer.writerow(["label","min","median","max","10th percentile","90th percentile","std dev","80th percentile",
                                    "70th percentile","NUM OF FACES"])
            for key in missed_labels:
                sdf_writer.writerow([str(key),*sdf_final_dict_list[str(key)]])





        print("Done writing sdf values")



        if(sdf_final_dict == [] and labels_list == []):
            print("NO CGAL DATA FOR " + str(neuron_ID))

            # deselect all
            bpy.ops.object.select_all(action='DESELECT')

            # selection
            #for ob in bpy.data.objects
            #bpy.data.objects[ob_name].select = True

            for obj in bpy.data.objects:
                if "neuron" in obj.name:
                    obj.select = True


            # remove it
            bpy.ops.object.delete() 
            ##########should this be a return??#########
            return

        print("getting cgal data--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()

        #complete_path = "/Users/brendancelii/Google Drive/Xaq Lab/Final_Blender/saved_sdf/sdf_saved_off.npz"
        #np.savez(complete_path,labels_list=labels_list,sdf_final_dict=sdf_final_dict)


        max_backbone_threshold = 200 #the absolute size if it is greater than this then labeled as a possible backbone
        backbone_threshold=40 #if the label meets the width requirements, these are the size requirements as well in order to be considered possible backbone
        shared_vert_threshold=10 #raw number of backbone verts that need to be shared in order for label to possibly be a backbone
        shared_vert_threshold_new = 5
        backbone_width_threshold = 0.10  #the median sdf/width value the segment has to have in order to be considered a possible backbone 
        #labels_list,verts_to_Face = smooth_backbone_vp3(labels_list,sdf_final_dict,backbone_width_threshold,max_backbone_threshold = max_backbone_threshold,backbone_threshold=backbone_threshold
        #        ,secondary_threshold=secondary_threshold,shared_vert_threshold=shared_vert_threshold,number_Flag = False, seg_numbers=1,smooth_Flag=True)

        old_labels_list = labels_list.copy()
        backbone_neighbor_min=10 # number of backbones in chain in order for label to keep backbone status
        labels_list,verts_to_Face = smooth_backbone_vp4(labels_list,sdf_final_dict,backbone_width_threshold,max_backbone_threshold = max_backbone_threshold,backbone_threshold=backbone_threshold,
                shared_vert_threshold=shared_vert_threshold,
                shared_vert_threshold_new = shared_vert_threshold_new,
                 backbone_neighbor_min=backbone_neighbor_min)

        #assemble the number of labels that are labeled backbone

        shaft_list = list(Counter(np.array(old_labels_list)[np.where(np.array(labels_list)=="backbone")[0].tolist()]).keys())

        shaft_list_final = [k for k in shaft_list if k not in missed_labels]

        with open(sdf_analysis_file[:-4] + "_STATS_TOTAL_shaft.csv",mode="w") as sdf_stat_file:
            sdf_writer = csv.writer(sdf_stat_file,delimiter=",")

            #[np.min(value),np.median(value),np.max(value),np.percentile(value,10),np.percentile(value,90),np.std(value)]
            sdf_writer.writerow(["label","min","median","max","10th percentile","90th percentile","std dev","80th percentile",
                                    "70th percentile","NUM OF FACES"])
            for key in shaft_list_final:
                sdf_writer.writerow([str(key),*sdf_final_dict_list[key]])

        print("smoothing backbone--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()

        #smooth away the stubs
        labels_list = filter_Stubs(labels_list,verts_to_Face,sdf_final_dict)

        print("about to rewrite the labels")
        for i in range(0,len(labels_list)):
            if labels_list[i] == "backbone":
                faces_raw[i].material_index = 0
            else:
                faces_raw[i].material_index = 2



        print("finished")
        print("--- %s seconds ---" % (time.time() - original_start_time))




In [10]:
import numpy as np
import math
from collections import Counter
import sys
#import matplotlib.pyplot as plt
import networkx as nx
import time
import csv
from pathlib import Path

import trimesh

class SpineClassifier(object):
    
    def __init__(self):
        self.mesh = None
    
    def get_spine_classification(self,file_location,file_name,clusters,smoothness):        
        original_start_time = time.time()    
        start_time = time.time()

        #initialize the object
        #file_location = "/notebooks/18_Spine_Allen"
        #file_name = "neuron-775959265587_part_2.off"
        full_path = str(Path(file_location) / Path(file_name))
        self.mesh = trimesh.load_mesh(full_path)
        
        
        #faces_raw = obj.data.polygons
        faces_raw = self.mesh.faces
        
        print(faces_raw)

In [12]:
file_name = "neuron-775959265587_part_2.off"
file_location = "/notebooks/18_Spine_Allen/neurons"

clusters = "20"
smoothness = "0.04"
 
myClassifier = SpineClassifier()
myClassifier.get_spine_classification(file_location,file_name,clusters,smoothness)

[[98607 43739 83871]
 [98607 83871 92015]
 [98607 92015 78666]
 ...
 [  937 16959  1501]
 [  937 20654 16959]
 [ 2760  4199  4194]]
